In [1]:
!pip install ripser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ripser: filename=ripser-0.6.4-cp37-cp37m-linux_x86_64.whl size=693994 sha256=c84dccc57fd2972d0aa23eebf411d4077223e20ac8c8ff8f336420a13ab1466c
  Stored in directory: /root/.cache/pip/wheels/c1/ce/06/8436a4c89b85a447e1355ffe6db64a321032ce003b81b2b35e
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18119 sha256=a8c1f0ec56b6d99598a1606f02bd3ab92d1156a831339548e7cb17be50a1e27c
  Stored in directory: /root/.cache/pip/wheels/de/c4/bb/56090fb5e4b0b436c8d677d055ac8ab6022aa6886019dd2990
Successfully built ripser hopcroftkarp


In [2]:
import torch
import torch.nn as nn
from torch.nn import Linear, GELU
from torch.nn.functional import relu
import torch.nn.functional as F
from torch import Tensor
import math


# Transformer classes
class Attention(nn.Module):
    # Single-head attention
    def __init__(self, embed_dim, num_heads, dropout = 0.0):
        super().__init__()
        attention_dim = embed_dim // num_heads

        self.WQ = nn.Linear(embed_dim, attention_dim, bias=False)
        self.WK = nn.Linear(embed_dim, attention_dim, bias=False)
        self.WV = nn.Linear(embed_dim, attention_dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        # query, key, value: (batch_size, length, embed_dim)
        # mask: (batch_size, length, length)
        Q = self.WQ(query)
        K = self.WK(key)
        V = self.WV(value)
        # Q, K, V: (batch_size, length, attention_dim)

        norm_factor = math.sqrt(Q.shape[-1])
        dot_products = torch.bmm(Q, K.transpose(1, 2)) / norm_factor
        # dot_products: (batch_size, length, length)
        if mask is not None:
            dot_products = dot_products.masked_fill(mask, -math.inf)
        attention_score = nn.functional.softmax(dot_products, dim=-1)
        attention = torch.bmm(self.dropout(attention_score), V)
        # attention_score: (batch_size, length, length)
        # attention: (batch_size, length, attention_dim)
        return attention, attention_score    

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout = 0.0):
        super(MultiHeadAttention, self).__init__()

        assert embed_dim % num_heads == 0
        self.attention_heads = nn.ModuleList([Attention(embed_dim, num_heads, dropout)
                                              for _ in range(num_heads)])

        self.linear = nn.Linear(embed_dim, embed_dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        # query, key, value: (batch_size, length, embed_dim)
        # mask: (batch_size, length, length)
        attentions, attention_scores = [], []

        for head in self.attention_heads:
            attention, attention_score = head(query, key, value, mask)
            attentions += [attention]
            attention_scores += [attention_score]

        attentions = torch.cat(attentions, dim=-1)
        attention_scores = torch.stack(attention_scores, dim=-1)
        # attentions: (batch_size, length, embed_dim)
        # attention_scores: (batch_size, length, length, num_heads)

        outputs = self.linear(attentions)
        outputs = self.dropout(outputs)
        # outputs: (batch_size, length, embed_dim)

        return outputs, attention_scores

class TransformerLayer(nn.Module):
    def __init__(self, embed_dim, fc_dim, num_heads, dropout = 0.0, activation = 'relu'):
        super().__init__()
        self.self_attention = MultiHeadAttention(embed_dim, num_heads, dropout)
        if activation == 'relu':
            self.feedforward = nn.Sequential(
                nn.Linear(embed_dim, fc_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(fc_dim, embed_dim),
                nn.Dropout(dropout)
            )
        elif activation == 'gelu':
            self.feedforward = nn.Sequential(
                nn.Linear(embed_dim, fc_dim),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(fc_dim, embed_dim),
                nn.Dropout(dropout)
            )
        else:
            raise NotImplementedError
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, inputs, mask = None):
        attention, attention_score = self.self_attention(query=inputs, key=inputs, 
                                                         value=inputs, mask=mask)
        outputs = inputs + attention
        outputs = self.norm1(outputs)
        outputs = outputs + self.feedforward(outputs)
        outputs = self.norm2(outputs)
        return outputs + inputs, attention_score    
    
def create_padding_mask(mask: Tensor):
    # tokens: (batch_size, length)
    length = mask.shape[-1]
    padding_mask = (mask == 0)
    padding_mask = padding_mask.unsqueeze(1).repeat(1, length, 1)
    # padding_mask: (batch_size, length, length)

    return padding_mask

class MLP(nn.Module):
    def __init__(self, dim_in: int, dim_out: int, width: int, nb_layers: int, skip=1, bias=True,
                 dim_in_2: int=None, modulation: str = '+'):
        """
        Args:
            dim_in: input dimension
            dim_out: output dimension
            width: hidden width
            nb_layers: number of layers
            skip: jump from residual connections
            bias: indicates presence of bias
            modulation (str): "+", "*" or "film". Used only if  dim_in_2 is not None (2 inputs to MLP)
        """
        super(MLP, self).__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.width = width
        self.nb_layers = nb_layers
        self.modulation = modulation
        self.hidden = nn.ModuleList()
        self.lin1 = nn.Linear(self.dim_in, width, bias)
        if dim_in_2 is not None:
            self.lin2 = nn.Linear(dim_in_2, width)
            if modulation == 'film':
                self.lin3 = nn.Linear(dim_in_2, width)
        self.skip = skip
        self.residual_start = dim_in == width
        self.residual_end = dim_out == width
        for i in range(nb_layers-2):
            self.hidden.append(nn.Linear(width, width, bias))
        self.lin_final = nn.Linear(width, dim_out, bias)

    def forward(self, x: Tensor, y: Tensor=None):
        """
        MLP is overloaded to be able to take two arguments.
        This is used in the first layer of the decoder to merge the set and the latent vector
        Args:
            x: a tensor with last dimension equals to dim_in
        """
        out = self.lin1(x)
        if y is not None:
            out2 = self.lin2(y)
            if self.modulation == '+':
                out = out + out2
            elif self.modulation == '*':
                out = out * out2
            elif self.modulation == 'film':
                out3 = self.lin3(y)
                out = out * torch.sigmoid(out2) + out3
            else:
                raise ValueError(f"Unknown modulation parameter: {self.modulation}")
        out = F.relu(out) + (x if self.residual_start else 0)
        for layer in self.hidden:
            out = out + layer(F.relu(out))
        out = self.lin_final(F.relu(out)) + (out if self.residual_end else 0)
        return out

class TransformerEncoder(nn.Module):
    def __init__(self, n_in, embed_dim, fc_dim, num_heads, num_layers, n_out_enc, dropout = 0.0, reduction="mean"):
        super(TransformerEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.reduction = reduction
        self.embedding = nn.Sequential(
            nn.Linear(n_in, embed_dim),
            nn.ReLU(),
        )
        # (batch, length, 3) -> (batch, length, emb_dim)
        
        
        self.query = nn.parameter.Parameter(
            torch.Tensor(1, embed_dim), requires_grad=True
        )
        self.scaled_dot_product_attention = MultiHeadAttention(embed_dim, num_heads, dropout)
        
        self.layers = nn.ModuleList([TransformerLayer(embed_dim, fc_dim, num_heads, dropout, 'relu') \
                                     for _ in range(num_layers)])
        
        self.output = nn.Linear(embed_dim, n_out_enc)
        
    def forward(self, X, mask):
        outputs = self.embedding(X) * math.sqrt(self.embed_dim)
        
        #attention_scores = []
        padding_mask = create_padding_mask(mask)
        for layer in self.layers:
            outputs, attention_score = layer(outputs, padding_mask)
        
        if self.reduction == "mean":
            lengths = mask.sum(dim=1).detach()
            outputs = (outputs * mask.unsqueeze(2)).sum(dim=1) / lengths.unsqueeze(1)
        elif self.reduction == "attention":
            outputs, _ = self.scaled_dot_product_attention(
                self.query.expand(outputs.shape[0], -1, -1),
                outputs,
                outputs,
                mask = padding_mask[:, 0, :].unsqueeze(1),
            )
            outputs = outputs.squeeze(dim=1)
        else:
            raise NotImplementedError
            
        # outputs : (batch_size, emb_dim)
        
        z = self.output(outputs)
        # z : (batch_size, n_out_enc)
        
        return z

In [3]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

import warnings
warnings.filterwarnings('ignore')

from ripser import lower_star_img
from ripser import Rips
vr = Rips()

import pickle
from tqdm import tqdm

import torch
from torch.nn import Linear
from torch.nn.functional import relu

from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from torch.optim.lr_scheduler import LinearLR, ReduceLROnPlateau, SequentialLR, CosineAnnealingWarmRestarts

Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [6]:
def generate_orbit(point_0, r, n=1000):
    
    X = np.zeros([n, 2])
    
    xcur, ycur = point_0[0], point_0[1]
    
    for idx in range(n):
        xcur = (xcur + r * ycur * (1. - ycur)) % 1
        ycur = (ycur + r * xcur * (1. - xcur)) % 1
        X[idx, :] = [xcur, ycur]
    
    return X

def generate_orbits(m, rs=[2.5, 3.5, 4.0, 4.1, 4.3], n=1000, random_state=None):
    
    # m orbits, each of n points of dimension 2
    orbits = np.zeros((m * len(rs), n, 2))
    
    # for each r
    for j, r in enumerate(rs):

        # initial points
        points_0 = random_state.uniform(size=(m,2))

        for i, point_0 in enumerate(points_0):
            orbits[j*m + i] = generate_orbit(points_0[i], rs[j])
            
    return orbits

In [7]:
random_state = np.random.RandomState(42)
X_orbit5k = generate_orbits(1000, random_state=random_state)
y = np.zeros(1000)

for i in range(1, 5):
    y = np.concatenate((y, np.ones(1000) * i))

In [8]:
def conv_pd(diagrams):
    pd = np.zeros((0, 3))

    for k, diagram_k in enumerate(diagrams):
        if k != 0:
            diagram_k = diagram_k[~np.isinf(diagram_k).any(axis=1)] # filter infs  
            diagram_k = np.concatenate((diagram_k, k * np.ones((diagram_k.shape[0], 1))), axis=1)
            if len(diagram_k) > 0:
                pd = np.concatenate((pd, diagram_k))           

    return pd

In [9]:
X = []

for x_pc in tqdm(X_orbit5k):
    diagram = conv_pd(vr.fit_transform(x_pc))
    diagram[:, 1] -= diagram[:, 0]
    X.append(diagram)

100%|██████████| 5000/5000 [1:32:43<00:00,  1.11s/it]  


In [10]:
class Orbit5kDataset(Dataset):
    
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
def collate_fn(data):
    
    tmp_pd, _ = data[0]
    
    n_batch = len(data)
    n_features_pd = tmp_pd.shape[1]
    n_points_pd = max(len(pd) for pd, _ in data)
    inputs_pd = np.zeros((n_batch, n_points_pd, n_features_pd), dtype=float)
    mask = np.zeros((n_batch, n_points_pd))
    labels = np.zeros(len(data))
    
    for i, (pd, label) in enumerate(data):
        inputs_pd[i][:len(pd)] = pd
        labels[i] = label
        mask[i][:len(pd)] = 1
    return torch.Tensor(inputs_pd), torch.Tensor(mask), torch.Tensor(labels).long()

In [13]:
class CustomPersformer(nn.Module):
    def __init__(self, n_in, embed_dim, fc_dim, num_heads, num_layers, n_out_enc, dropout = 0.1, reduction="mean"):
        super(CustomPersformer, self).__init__()
        self.embed_dim = embed_dim
        self.reduction = reduction
        self.embedding = nn.Sequential(
            nn.Linear(n_in, embed_dim),
            nn.GELU(),
        )
        # (batch, length, 3) -> (batch, length, emb_dim)
        
        
        self.query = nn.parameter.Parameter(
            torch.Tensor(1, embed_dim), requires_grad=True
        )
        self.scaled_dot_product_attention = MultiHeadAttention(embed_dim, num_heads, dropout)
        
        self.layers = nn.ModuleList([TransformerLayer(embed_dim, fc_dim, num_heads, dropout, 'gelu') \
                                     for _ in range(num_layers)])
        
        self.output = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, 256),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(64, n_out_enc),
        )
        
    def forward(self, X, mask):
        outputs = self.embedding(X) * math.sqrt(self.embed_dim)
        
        #attention_scores = []
        #print(mask.shape)
        padding_mask = create_padding_mask(mask)
        for layer in self.layers:
            outputs, attention_score = layer(outputs, padding_mask)
        
        if self.reduction == "mean":
            lengths = mask.sum(dim=1).detach()
            outputs = (outputs * mask.unsqueeze(2)).sum(dim=1) / lengths.unsqueeze(1)
        elif self.reduction == "attention":
            #print(padding_mask.shape)
            outputs, _ = self.scaled_dot_product_attention(
                self.query.expand(outputs.shape[0], -1, -1),
                outputs,
                outputs,
                mask = padding_mask[:, 0, :].unsqueeze(1),
            )
            outputs = outputs.squeeze(dim=1)
        else:
            raise NotImplementedError
            
        # outputs : (batch_size, emb_dim)
        
        z = self.output(outputs)
        # z : (batch_size, n_out_enc)
        
        return z

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

n_repeats = 1
batch_size = 32
lr = 0.001
warmup_iters = 10
n_train, n_test = 3500, 1500
n_epochs = 990 + warmup_iters


history = np.zeros((n_repeats, n_epochs, 3))
criterion = CrossEntropyLoss()

dataset = Orbit5kDataset(X, y)
    
# data init
dataset_train, dataset_test = random_split(dataset, [n_train, n_test], generator=torch.Generator().manual_seed(54))
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dataloader_test =  DataLoader(dataset_test, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


# Initialize the model
model = CustomPersformer(n_in=3, embed_dim=64, fc_dim=128, num_heads=8, num_layers=5, n_out_enc=5, dropout=0.0, 
                           reduction="attention").to(device)


optimizer = AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler1 = LinearLR(optimizer, start_factor=0.00005, total_iters=warmup_iters)

print("{:3} {:6} {:6} {:6}".format(1, "Loss", "Train", "Test"))

for epoch_idx in range(n_epochs):

    # train
    model.train()

    loss_epoch = []
    for batch in tqdm(dataloader_train):
        loss_batch = criterion(model(batch[0].to(device), batch[1].to(device)), batch[2].to(device))
        loss_batch.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_epoch.append(loss_batch.detach().cpu())

    loss_epoch_mean = np.array(loss_epoch).mean()

    # test
    model.eval()

    correct = 0
    for batch in dataloader_train:
        y_hat = model(batch[0].to(device), batch[1].to(device)).argmax(dim=1)
        correct += int((y_hat == batch[2].to(device)).sum())
    accuracy_train = correct / len(dataloader_train.dataset)

    correct = 0
    for batch in dataloader_test:
        y_hat = model(batch[0].to(device), batch[1].to(device)).argmax(dim=1)
        correct += int((y_hat == batch[2].to(device)).sum())
    accuracy_test = correct / len(dataloader_test.dataset)

    loss_epoch_test = []
    for batch in tqdm(dataloader_test):
        loss_batch = criterion(model(batch[0].to(device), batch[1].to(device)), batch[2].to(device))
        loss_epoch_test.append(loss_batch.detach().cpu())

    loss_epoch_mean_test = np.array(loss_epoch_test).mean()

    if epoch_idx < warmup_iters:
        scheduler1.step()
    else:
        if epoch_idx == warmup_iters:
            scheduler2 = CosineAnnealingWarmRestarts(optimizer, 330)
        #scheduler2.step(loss_epoch_mean_test)
        scheduler2.step()

    print("{:3} {:.4f} {:.4f} {:.4f}".format(epoch_idx, loss_epoch_mean, accuracy_train, accuracy_test))

  1 Loss   Train  Test  


100%|██████████| 47/47 [00:01<00:00, 33.49it/s]


  0 1.6128 0.1997 0.2007


100%|██████████| 47/47 [00:01<00:00, 33.35it/s]


  1 1.6113 0.1980 0.2047


...

398 0.2589 0.9009 0.8880


100%|██████████| 47/47 [00:01<00:00, 32.50it/s]


399 0.2476 0.9097 0.9007


 35%|███▌      | 39/110 [00:03<00:06, 11.32it/s]


KeyboardInterrupt: 

In [20]:
for epoch_idx in range(400, 669):

    # train
    model.train()

    loss_epoch = []
    for batch in tqdm(dataloader_train):
        loss_batch = criterion(model(batch[0].to(device), batch[1].to(device)), batch[2].to(device))
        loss_batch.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_epoch.append(loss_batch.detach().cpu())

    loss_epoch_mean = np.array(loss_epoch).mean()

    # test
    model.eval()

    correct = 0
    for batch in dataloader_train:
        y_hat = model(batch[0].to(device), batch[1].to(device)).argmax(dim=1)
        correct += int((y_hat == batch[2].to(device)).sum())
    accuracy_train = correct / len(dataloader_train.dataset)

    correct = 0
    for batch in dataloader_test:
        y_hat = model(batch[0].to(device), batch[1].to(device)).argmax(dim=1)
        correct += int((y_hat == batch[2].to(device)).sum())
    accuracy_test = correct / len(dataloader_test.dataset)

    loss_epoch_test = []
    for batch in tqdm(dataloader_test):
        loss_batch = criterion(model(batch[0].to(device), batch[1].to(device)), batch[2].to(device))
        loss_epoch_test.append(loss_batch.detach().cpu())

    loss_epoch_mean_test = np.array(loss_epoch_test).mean()

    if epoch_idx < warmup_iters:
        scheduler1.step()
    else:
        if epoch_idx == warmup_iters:
            scheduler2 = CosineAnnealingWarmRestarts(optimizer, 330)
        #scheduler2.step(loss_epoch_mean_test)
        scheduler2.step()

    print("{:3} {:.4f} {:.4f} {:.4f}".format(epoch_idx, loss_epoch_mean, accuracy_train, accuracy_test))

100%|██████████| 47/47 [00:01<00:00, 33.02it/s]


400 0.2451 0.8963 0.8667


100%|██████████| 47/47 [00:01<00:00, 32.43it/s]


401 0.2404 0.9054 0.8820


100%|██████████| 47/47 [00:01<00:00, 33.03it/s]


...

499 0.1733 0.9263 0.8860


100%|██████████| 47/47 [00:01<00:00, 32.82it/s]


500 0.1735 0.9297 0.8747


 12%|█▏        | 13/110 [00:01<00:09, 10.50it/s]


KeyboardInterrupt: 

In [15]:
torch.save(model.state_dict(), "persformer_orbit5k_90acc.pt")

In [17]:
torch.save(dataloader_train, 'dataloader_train.pt')

In [18]:
torch.save(dataloader_test, 'dataloader_test.pt')